In [ ]:
%pip install "psycopg[binary,pool]" pytz requests


In [ ]:
import requests
import json

# download data
data = {}


# returns JSON

url = "https://api.open-meteo.com/v1/forecast?latitude=47.712596&longitude=13.618029&hourly=temperature_2m,cloud_cover,rain,snowfall,snow_depth,cloud_cover_low,cloud_cover_mid,cloud_cover_high,visibility,relative_humidity_2m,dew_point_2m&timezone=Europe%2FBerlin&forecast_days=14"
response = requests.get(url)

if response.status_code == 200:
    data = response.json()  # Automatically parses JSON into a Python dictionary
    print("Downloaded JSON successfully!")
    print(data)
else:
    print(f"Failed to download JSON. Status code: {response.status_code}")


In [ ]:
hourly_units = data['hourly_units']
hourly_units


Load CSV into a Python data structure for processing

In [ ]:
metrics = list(hourly_units.keys())
metrics

In [ ]:
import pandas as pd

In [ ]:
hourly = data["hourly"]
data_frame_setup = {}

for metric in metrics:
    data_frame_setup[metric] = hourly[metric]
    

In [ ]:
df = pd.DataFrame(data_frame_setup)

df['time'] = pd.to_datetime(df['time'])



In [ ]:
df.describe()

In [ ]:
from psycopg import connect
from psycopg.rows import dict_row
from psycopg import sql



os.environ['PGSERVICEFILE'] = "/home/martin/Workspace/Energiegemeinschaft/notebooks/.pg_service.conf"
os.environ['PGPASSFILE'] = "/home/martin/Workspace/Energiegemeinschaft/notebooks/.pgpass"

data_tuples = [
    (
        row['time'],
        row['temperature_2m'],
        row['cloud_cover'],
        row['rain'],
        row['snowfall'],
        row['snow_depth'],
        row['cloud_cover_low'],
        row['cloud_cover_mid'],
        row['cloud_cover_high'],
        row['visibility'],
        row['relative_humidity_2m'],
        row['dew_point_2m'],
    )
    for _, row in df.iterrows()
]

with psycopg.connect(service='eeg-middleware') as conn:
    with conn.cursor() as cur:
        
        cur.executemany(
            """
            INSERT INTO weather_weatherdata (
                time, temperature_2m, cloud_cover, rain, snowfall, snow_depth,
                cloud_cover_low, cloud_cover_mid, cloud_cover_high,
                visibility, relative_humidity_2m, dew_point_2m
            ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
            ON CONFLICT (time) DO UPDATE SET
                temperature_2m = EXCLUDED.temperature_2m,
                cloud_cover = EXCLUDED.cloud_cover,
                rain = EXCLUDED.rain,
                snowfall = EXCLUDED.snowfall,
                snow_depth = EXCLUDED.snow_depth,
                cloud_cover_low = EXCLUDED.cloud_cover_low,
                cloud_cover_mid = EXCLUDED.cloud_cover_mid,
                cloud_cover_high = EXCLUDED.cloud_cover_high,
                visibility = EXCLUDED.visibility,
                relative_humidity_2m = EXCLUDED.relative_humidity_2m,
                dew_point_2m = EXCLUDED.dew_point_2m
            """,
            data_tuples
        )
        print("Data inserted successfully.")
        
        